# Guía estructurada de la Tarea 1

Este notebook resume los hitos principales del enunciado para mantener un plan de trabajo ordenado. Avanza sección por sección y completa cada hito conforme desarrolles tu solución del proyecto.


In [ ]:
# Configuración base para reproducibilidad
GLOBAL_SEED = 42
DATASET_ID = "<por_definir>"


## 1. E–T–P y framing

- Describe la Experiencia (E), la Tarea (T) y el Performance/criterio (P).
- Define el framing principal como clasificación binaria de `purchase` con salida probabilística.
- Justifica si usarás tasas agregadas y la decisión de negocio basada en umbrales.



## 2. Métricas y pérdida

- Usa log-loss (entropía cruzada) como pérdida principal.
- Reporta AUC y Brier score.
- Explica por qué MSE no es adecuado como objetivo principal y relaciónalo con máxima verosimilitud.



## 3. Diseño de validación y control de capacidad

- Define particiones train/valid/test (70/15/15) o k-fold para escoger hiperparámetros.
- Reentrena con los mejores hiperparámetros antes del test.
- Controla capacidad con regularización L2 (parámetro C) y grafica curvas train/valid vs. complejidad o learning curves.
- Explica el trade-off sesgo–varianza.



## 4. Preprocesamiento

- Explora el dataset para identificar duplicados, variables irrelevantes o leakage.
- Define codificación de variables categóricas, escalamiento y manejo de outliers o nulos según corresponda.



## 5. Modelado predictivo

- Entrena al menos dos modelos de clasificación (ej. regresión logística, árbol, random forest, XGBoost).
- Compara su desempeño inicial.



## 6. Evaluación

- Reporta métricas de clasificación: accuracy, precision, recall, F1-score y AUC-ROC.



## 7. Discusión de resultados

- Destaca variables relevantes para los modelos.
- Justifica columnas excluidas (especialmente leaks).
- Propón insights accionables para la empresa.



## 8. Política operativa y sensibilidad

- Formula una regla clara: contactar/ofrecer si la probabilidad estimada supera el umbral t.
- Analiza sensibilidad (ej. utilidad esperada por umbral) y discute implicancias.



## 9. Riesgos y mitigación

- Identifica al menos tres riesgos: leakage, sesgo de muestreo, shift temporal/segmento.
- Propón mitigaciones (auditoría de variables, validación por segmento o fuera de tiempo, calibration, A/B).



## 10. Resultados y conclusiones

- Resume hallazgos clave: saturación, desempeño en test, umbral recomendado.
- Indica cómo debe operar la empresa con el modelo final.



## Extras opcionales

- Mostrar efectos de usar `leak_after_offer` para evidenciar data leakage.
- Calcular métricas de negocio (ej. expected profit) usando `unit_margin_if_buy`.
- Comparar `discount` numérico vs. `discount_bucket`.



## Formato y reproducibilidad

- Declara semilla global y DATASET_ID en la primera celda.
- Fija semillas para NumPy/sklearn y repórtalas.
- Reporta versiones de librerías y hash SHA-256 del CSV.
- Trabaja sólo con el dataset entregado, sin regenerar datos.
- Entrega resultados específicos de tu dataset.



# esto es una prueba